In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
# 아이디/비밀번호
id = ''
password = ''

# 년도 선택
year = '2022'

# 전체 분기별

# 메뉴별 분기

In [45]:
chrome_driver = './chromedriver.exe'
login_url = 'https://ceo.baemin.com/web/login?returnUrl=https%3A%2F%2Fceo.baemin.com%2F'
url = 'https://ceo.baemin.com/self-service/orders/history'
DF = pd.DataFrame()

#가게 선택
            # 냉면, 국밥, 치밥, 타코야끼
stores = ['13260990', '13643240', '13442351', '13708011']
store = stores[2] # 치밥 선택

# 로그인 페이지 이동
driver = webdriver.Chrome(chrome_driver)
driver.get(login_url)
time.sleep(1)

# 로그인
ID = driver.find_element(by='name', value='id')
PASS = driver.find_element(by='name', value='password')
ID.send_keys(id)
PASS.send_keys(password)
button = driver.find_element(by='xpath', value='//*[@class="Button__StyledButton-sc-394bsp-0 IqKSG"]')
button.click()
time.sleep(1)

# 페이지 이동
driver.get(url)
time.sleep(2)

# 가게 선택
driver.find_element(by='xpath', value='//*[@class="FilterContainer-module__ZmLA"]').click()
time.sleep(1)
driver.find_element(by='xpath', value=f'//select/option[@value="{store}"]').click()
driver.find_element(by='xpath', value='//button[@class="button medium"]').click()
time.sleep(1)

for i in range(6, 7): # 하반기
    try:
        data = []
        
        # 기간 선택
        driver.find_element(by='xpath', value='//*[@class="FilterContainer-module__dOxp FilterContainer-module__dam2"]').click()
        time.sleep(1)
        driver.find_element(by='xpath', value='//div[@class="Radio-module__P8H0"]/label[3]/p').click() # 분기
        driver.find_element(by='xpath', value=f'//select[@class="YearSelector-module__DaIR"]/option[@value="{year}"]').click() # 년도
        driver.find_element(by='css selector', value=f'.form-control > div:nth-child({i}) input').click() # 상/하반기
        driver.find_element(by='xpath', value='//button[@class="button medium"]').click()
        time.sleep(1)
        
        while True:
            try:
                # 모두 펼쳐보기
                button_open = driver.find_element(by='xpath', value='//*[@class="button ExpandAllButton-module__sp85 small text"]')
                button_open.click()
                time.sleep(2)

                # scrapping
                soup = BeautifulSoup(driver.page_source,'html.parser')
                texts = soup.find_all('td', attrs={"class":"pivoted"})
                

                for i in range(0, 100, 10):
                    menu = texts[i+9].find('div', attrs={"class":"DetailInfo-module__DjeY"}).text
                    a = [texts[i].text[5:], texts[i+1].text[:12], texts[i+1].text[18:20], texts[i+1].text[21:], menu.split('총 결제금액')[0], texts[i+6].text, texts[i+7].text[:-1].replace(',', '')]
                    data.append(a)
                    
                # 다음 페이지
                button_next = driver.find_element(by='xpath', value='//a[@class="Paging-module__BwVC"]/i[@class="icon icon-chevron-right"]')
                button_next.click()
                time.sleep(1)
            except:
                print(len(data))
                break
    except:
        print('기간 선택에 대한 에러')
        pass

    df = pd.DataFrame(data, columns=['주문번호', '주문일자', '오전/오후', '주문시간', '메뉴', '배달/포장 구분', '주문금액'])
    DF = pd.concat([df, DF], ignore_index=True)
    DF.to_csv(f'../data/data_{store}_{year}.csv')

driver.quit()
print('종료')

C:\Users\Eugene\AppData\Local\Temp\ipykernel_21512\2649644680.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver)


180
종료
